## Imports

In [2]:
# Install needed packages first:
# pip install langchain langchain-community openai pandas
# from langchain import LLMChain, PromptTemplate
# from langchain.chat_models import ChatOpenAI
# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.prompts import PromptTemplate
# from langchain_openai import OpenAI

from langchain.tools import tool
from langchain.chat_models import init_chat_model

from langchain_community.document_loaders import PyPDFLoader

import pandas as pd
import json
import re
import numpy as np


from langchain.tools import tool
from langchain.agents import create_agent

### Parser Tools

In [3]:
@tool
def parse_pdf(file_path: str) -> str:
    """Load PDF and return raw text."""
    loader = PyPDFLoader(file_path)
    docs = loader.load()
    full_text = "\n".join([doc.page_content for doc in docs])
    return full_text

@tool
def parse_excel(files_path: str) -> str:
    """Load Excel and return JSON-string of rows."""
    df = pd.read_csv("statements\oct2025_csv.csv", header=None)
    info_df = df.iloc[:20]
    info_df = info_df.drop(columns=[2,3,4,5,6])
    info_text = info_df.to_string(index=False, header=False)

    # Clean the text:
    info_text = "\n".join(
        re.sub(r'\s+', ' ', line.strip())   
        for line in info_text.splitlines()
        if line.strip()                    
    )

    transaction_df = df.iloc[21:].copy() # data starts from row 21 downward
    headers = [str(col).strip() for col in df.iloc[20].tolist()]
    transaction_df.columns = headers

    transaction_df = transaction_df.drop(columns=["Value Date"])

    # Replace empty strings or whitespace-only cells with NaN
    transaction_df['Credit'] = transaction_df['Credit'].replace(r'^\s*$', np.nan, regex=True)
    transaction_df['Debit'] = transaction_df['Debit'].replace(r'^\s*$', np.nan, regex=True)

    transaction_df = transaction_df.rename(columns={"Txn Date": "Date", "Ref No./Cheque No.": "Reference Number"})

    transaction_df["Type"] = np.where(
        transaction_df['Credit'].notna(), 'Income',
        np.where(transaction_df['Debit'].notna(), 'Expense', 'Unknown')
    )

    trans_json = transaction_df.to_json(orient="records")

    return info_text, trans_json

### Agent

In [26]:
categorize_schema = {
    "type": "object",
    "description": "Structured output containing categorized bank transactions.",
    "properties": {
        "overview": {"type": "string", "description": (
            "Basic details of the statement file, including the account holder's name, "
            "account number, address, opening balance, and closing balance, etc. along with a brief overview of the account holder."
        )},
        "transactions": {
            "type": "array",
            "description": "List of categorized transactions.",
            "items": {
                "type": "object",
                "properties": {
                    "date": {"type": "string", "description": "Transaction date."},
                    "description": {"type": "string", "description": "Narration or details."},
                    "amount": {"type": "number", "description": "Positive for income, negative for expense."},
                    "type": {
                        "type": "string",
                        "enum": ["income", "expense"],
                        "description": "Transaction type."
                    },
                    "category": {
                        "type": "string",
                        "description": (
                            "For income: one of ['salary', 'investment redemption', 'dividend', 'interest', 'other']; "
                            "for expense: one of ['groceries', 'rent', 'food', 'EMI', 'investment', 'utilities', 'travel', "
                            "'entertainment', 'medical', 'other']."
                        )
                    }
                },
                "required": ["date", "description", "amount", "type", "category"]
            }
        },
        "insights": {"type": "string", "description": (
            "Insights on spending patterns, savings rate, unusual transactions, and "
            "personalized financial recommendations for the account holder."
        )}
    },
    "required": ["overview", "transactions", "insights"]
}



agent = create_agent(
    model="openai:gpt-5-nano",
    tools=[parse_excel, parse_pdf],
    system_prompt=(
        "You are a financial data assistant. You will be given either an Excel or PDF file path. Use the correct tool "
        "(parse_excel or parse_pdf) to extract transaction details. From the extracted data, identify all the transactions "
        "and output them in the specified response schema. Each transaction must include date, description, amount, type "
        "(“income” or “expense”), and a clear category. Use appropriate categories (salary, rent, groceries, utilities, etc.) "
        "and avoid using “other” unless absolutely necessary. Do not leave any transaction uncategorized or missing."
    ),
    response_format=categorize_schema  # Auto-selects ProviderStrategy
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "statements\Mock_Bank_Statement.pdf"}]
})

result["structured_response"]

{'overview': 'Account: Rahul Verma | Bank: State Bank of India | Branch: Kormangala, Bangalore | Account Number: 0000005023841123 | Statement period: 01-Apr-2024 to 31-Aug-2024 | Closing balance: 326,800. Opening balance not shown in the extracted data. The statement shows regular salary credits from Infosys and recurring expenses across groceries, utilities, rent, travel, entertainment, and EMI payments. A refund of 3,000 was received on 21-May-2024 ( Indigo Airlines ). Overall, spending leans toward discretionary categories (food, entertainment, travel) with substantial EMI payments and monthly rent; salary deposits provide liquidity for savings, culminating in a closing balance of 326,800.',
 'transactions': [{'date': '2024-04-01',
   'description': 'Salary Credit – INFOSYS LTD',
   'amount': 120000,
   'type': 'income',
   'category': 'salary'},
  {'date': '2024-04-03',
   'description': 'AMAZON Purchase – Groceries',
   'amount': -3200,
   'type': 'expense',
   'category': 'grocer

In [37]:
print(result)

{'messages': [HumanMessage(content='statements\\Mock_Bank_Statement.pdf', additional_kwargs={}, response_metadata={}, id='17b26a07-994a-464e-9485-64da20829285'), AIMessage(content='', additional_kwargs={'parsed': None, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 160, 'prompt_tokens': 534, 'total_tokens': 694, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CWFuSY7RrlaE2e57daC54cv32EDW4', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--b7d94433-849e-4345-b412-db64af3a9cf5-0', tool_calls=[{'name': 'parse_pdf', 'args': {'file_path': 'statements\\Mock_Bank_Statement.pdf'}, 'id': 'call_NO2QkPyd2wjY9nLzhYHXXXzc', 'type': 'tool_call'}], usage_metadata={'inp

In [27]:
head_results = result["structured_response"]["transactions"]
print("Total number of analysed transactions:",len(head_results))
for each_result in head_results:
    print(each_result)

Total number of analysed transactions: 52
{'date': '2024-04-01', 'description': 'Salary Credit – INFOSYS LTD', 'amount': 120000, 'type': 'income', 'category': 'salary'}
{'date': '2024-04-03', 'description': 'AMAZON Purchase – Groceries', 'amount': -3200, 'type': 'expense', 'category': 'groceries'}
{'date': '2024-04-05', 'description': 'EMI Payment – HDFC BANK Loan No. 8734', 'amount': -12500, 'type': 'expense', 'category': 'EMI'}
{'date': '2024-04-07', 'description': 'ZOMATO – Food Order', 'amount': -2400, 'type': 'expense', 'category': 'food'}
{'date': '2024-04-10', 'description': 'Swiggy – Food Delivery', 'amount': -1500, 'type': 'expense', 'category': 'food'}
{'date': '2024-04-12', 'description': 'Interest on FD – SBI', 'amount': 2000, 'type': 'income', 'category': 'interest'}
{'date': '2024-04-15', 'description': 'IRCTC – Train Ticket (Delhi Trip)', 'amount': -4800, 'type': 'expense', 'category': 'travel'}
{'date': '2024-04-18', 'description': 'Fuel Payment – HP Petrol Pump', 'amou

In [28]:
print("Overview:", result["structured_response"]["overview"])
print("Insights:", result["structured_response"]["insights"])

Overview: Account: Rahul Verma | Bank: State Bank of India | Branch: Kormangala, Bangalore | Account Number: 0000005023841123 | Statement period: 01-Apr-2024 to 31-Aug-2024 | Closing balance: 326,800. Opening balance not shown in the extracted data. The statement shows regular salary credits from Infosys and recurring expenses across groceries, utilities, rent, travel, entertainment, and EMI payments. A refund of 3,000 was received on 21-May-2024 ( Indigo Airlines ). Overall, spending leans toward discretionary categories (food, entertainment, travel) with substantial EMI payments and monthly rent; salary deposits provide liquidity for savings, culminating in a closing balance of 326,800.
Insights: Key patterns: Regular monthly salary deposits of 120,000 on the 1st (and occasionally other months), with EMI payments (~12,500 monthly) and rent of 15,000 every month. Discretionary spending is prominent in food/entertainment (Swiggy, Zomato, Netflix, cinema, dining out, mall shopping). Tra

In [29]:
out_df = pd.DataFrame(result["structured_response"]["transactions"])
out_df["date"] = pd.to_datetime(out_df["date"])

# Create a readable month label (e.g., "October 2025")
out_df["month"] = out_df["date"].dt.strftime("%B %Y")

out_df

,date,description,amount,type,category,month
0,2024-04-01,Salary Credit – INFOSYS LTD,120000,income,salary,April 2024
1,2024-04-03,AMAZON Purchase – Groceries,-3200,expense,groceries,April 2024
2,2024-04-05,EMI Payment – HDFC BANK Loan No. 8734,-12500,expense,EMI,April 2024
3,2024-04-07,ZOMATO – Food Order,-2400,expense,food,April 2024
4,2024-04-10,Swiggy – Food Delivery,-1500,expense,food,April 2024
5,2024-04-12,Interest on FD – SBI,2000,income,interest,April 2024
6,2024-04-15,IRCTC – Train Ticket (Delhi Trip),-4800,expense,travel,April 2024
7,2024-04-18,Fuel Payment – HP Petrol Pump,-5000,expense,travel,April 2024
8,2024-04-20,Rent Transfer – To ANKIT SHARMA,-15000,expense,rent,April 2024
9,2024-04-23,Netflix Annual Subscription,-3000,expense,entertainment,April 2024


#### Monthly summary of income, expenses and savings

In [30]:
# Then group and summarize
monthly_summary = (
    out_df.groupby(["month", "type"])["amount"]
      .sum()
      .unstack(fill_value=0)
      .reset_index()
      .rename_axis(None, axis=1)
)

monthly_summary["savings"] = monthly_summary.get("income", 0) + monthly_summary.get("expense", 0)

monthly_summary



,month,expense,income,savings
0,April 2024,-47400,122000,74600
1,August 2024,-39600,120000,80400
2,July 2024,-62100,120000,57900
3,June 2024,-60400,120000,59600
4,May 2024,-55100,123000,67900


In [31]:
total_income = monthly_summary["income"].sum()
total_expense = monthly_summary["expense"].sum()
total_savings = monthly_summary["savings"].sum()


print("Total Income:", total_income)
print("Total Expense:", total_expense)
print("Total Savings:", total_savings)

Total Income: 605000
Total Expense: -264600
Total Savings: 340400


#### Category Summary

In [32]:
category_summary = (
    out_df.groupby(["category", "type"])["amount"]
      .sum()
      .unstack(fill_value=0)
      .reset_index()
      .rename_axis(None, axis=1)
)

category_summary

,category,expense,income
0,EMI,-71700,0
1,entertainment,-26900,0
2,food,-19700,0
3,groceries,-30800,0
4,interest,0,2000
5,medical,-10000,0
6,other,0,3000
7,rent,-65000,0
8,salary,0,600000
9,travel,-31600,0


## Experimenting

In [39]:
df = pd.read_csv("statements\oct2025_csv.csv", header=None)
info_df = df.iloc[:20]
info_df = info_df.drop(columns=[2,3,4,5,6])
info_text = info_df.to_string(index=False, header=False)

# Clean the text:
info_text = "\n".join(
    re.sub(r'\s+', ' ', line.strip())   # collapse multiple spaces → single space
    for line in info_text.splitlines()
    if line.strip()                     # skip blank lines
)
# You can then print it:
print(info_text)
# info_df

transaction_df = df.iloc[21:].copy()            # data starts from row 21 downward
headers = [str(col).strip() for col in df.iloc[20].tolist()]
transaction_df.columns = headers

transaction_df = transaction_df.drop(columns=["Value Date"])

# Replace empty strings or whitespace-only cells with NaN
transaction_df['Credit'] = transaction_df['Credit'].replace(r'^\s*$', np.nan, regex=True)
transaction_df['Debit'] = transaction_df['Debit'].replace(r'^\s*$', np.nan, regex=True)

transaction_df = transaction_df.rename(columns={"Txn Date": "Date", "Ref No./Cheque No.": "Reference Number"})

transaction_df["Type"] = np.where(
    transaction_df['Credit'].notna(), 'Income',
    np.where(transaction_df['Debit'].notna(), 'Expense', 'Unknown')
)
# transaction_df
trans_json = transaction_df.to_json(orient="records")

print(trans_json)

Account Name : Mr. Rohit Patnaik
Address : C/O Sasi Bhusan Patnaik, 00, WRITER STRE
ET
Bishamakatak-765019
396:Rayagada
Date : 28-Oct-25
Account Number : _00000040381454973
Account Description: LOTUS SAVING BANK-ADHAR- NCHQ
Branch : BISSAMCUTTACK
Drawing Power : 0
Interest Rate(% p.a.): 2.5
MOD Balance : 0
CIF No. : _90805421606
IFS (Indian Financial System) Code : SBIN0012100
CKYCR Number : XXXXXXXXXX1294
MICR (Magnetic Ink Character Recognition) Code : _765002501
Nomination Registered : No
Balance on 1 Oct 2025 : 8,413.30
Start Date : 01-Oct-25
End Date : 31-Oct-25
[{"Date":"01-Oct-25","Description":"   TO TRANSFER-UPI\/DR\/527449242189\/RAJENDRA\/YESB\/q737733319\/lassi--","Reference Number":"TRANSFER TO 4897693162093","Debit":"60","Credit":null,"Balance":"8,353.30","Type":"Expense"},{"Date":"01-Oct-25","Description":"   TO TRANSFER-UPI\/DR\/527457715352\/SABINA B\/YESB\/q72025260@\/bapa--","Reference Number":"TRANSFER TO 4897693162093","Debit":"400","Credit":null,"Balance":"7,953.3